In [0]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
import pandas as pd
import numpy as np

In [0]:
table = pd.read_csv('quora_question_pairs_rus.csv')

In [7]:
table.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
test_N = 5000

In [0]:
test_table = table.head(test_N)

In [0]:
texts = []
for row in test_table['question2']:
  texts.append(str(row))  

In [0]:
queries = []
for row in test_table['question1']:
  queries.append(str(row))  

In [14]:
pip install pymorphy2

In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
import re
def tokenize(line): #функция возвращает список токенов данного предложения
  ws = []
  words = line.split()
  for w in words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—–"…}{/\|«»>]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

FASTTEXT

In [0]:
# если модель без тэгов
model_file = 'model.model'

#model = Word2Vec.load(model_file)

In [18]:
model = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
query = 'Черепаха победила зайца'

In [20]:
query_tok = tokenize(query)
query_tok

['черепаха', 'победить', 'заяц']

In [21]:
for i in query_tok:
  res = i in model.wv
  print(i, ': ', res)
  print(model[i][:5])

черепаха :  True
[ 0.38691303 -0.01353064  0.47982895  0.24796614 -0.4995282 ]
победить :  True
[ 0.19871868  0.1151659  -0.29912788  0.18883215  0.1581482 ]
заяц :  True
[ 0.35733244  0.25698143  0.3300038   0.34465954 -0.43749848]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
def doc_vector(doc):

  lemmas = tokenize(doc)
  
  lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
  doc_vec = np.zeros((model.vector_size,))
  
  for idx, lemma in enumerate(lemmas):
    if lemma in model.vocab: #добавим try-except, так как при тестировании появлялась ошибка AttributeError
      try:
        lemmas_vectors[idx] = model[lemma]
      except AttributeError as err:
        print(err)
  if lemmas_vectors.shape[0] is not 0:
    doc_vec = np.mean(lemmas_vectors, axis=0)
    return doc_vec

In [0]:
result = doc_vector(query)

In [24]:
result[:10]

array([ 0.31432138,  0.1195389 ,  0.17023496,  0.26048594, -0.25962616,
        0.19820209,  0.03645031,  0.03666434, -0.05090792,  0.37595434])

In [25]:
from time import time
start1 = time()
vecs = []
for i in texts:
  res = doc_vector(i)
  vecs.append(res)
time1 = str(time() - start1)
print('Time of the first function: ', time1)
print('Текст: ', texts[0])
print('Его вектор: ', vecs[0][:5])

Time of the first function:  9.861446380615234
Текст:  что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад
Его вектор:  [ 0.12532789  0.04991495  0.18384925  0.03815691 -0.00037019]


In [0]:
def search_fasttext(query, n):
  cos_sims = []
  for doc in vecs:
    res = np.inner(query, doc)/(np.linalg.norm(query)*np.linalg.norm(doc))
    cos_sims.append(res)
  ids = []
  for x in sorted(enumerate(cos_sims), key=lambda x:x[1], reverse=True):
    ids.append(x[0])
  top_n = ids[:n]
  results = []
  for i in ids:
    results.append([i, np.array(texts)[i], cos_sims[i]])
  return results

Проверю точность только для 10 примеров, так как долго работает

In [0]:
query_vecs = []
for i in queries[:10]:
  res = doc_vector(i)
  query_vecs.append(res)

In [62]:
results = []
for i in query_vecs:
  res = search_fasttext(i, 5)
  results.append(res)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [63]:
results

[[[1165, 'какой лучший подкаст в истории', 0.7925937212792524],
  [2572, 'каковы самые большие ошибки в истории физики', 0.7706777512060804],
  [2258,
   'какова ваша самая удивительная личная история о путешествии в Филиппины',
   0.7452648365037052],
  [1388,
   'Каковы интересные приключенческие дела в Гидерабаде?',
   0.7296957193734224],
  [4447, 'Каков ваш обзор истории Чу Юань', 0.7089879463370574],
  [1421, 'каковы некоторые теории жизни', 0.7054178313560893],
  [1305, 'Какова цель жизни', 0.69312442219178],
  [2064, 'какая самая большая афера в истории', 0.6923792866992001],
  [621, 'Какова цель науки?', 0.6904714550955345],
  [3498,
   'как я могу создать и провести в нашей истории рассказ о жизни в snapchat, с кем я могу связаться в snapchat, чтобы начать живую историю',
   0.6808299959971958],
  [4180, 'каковы лучшие драматические телешоу', 0.6784570359661716],
  [1182, 'Какова ваша миссия в жизни?', 0.678352517920186],
  [817,
   'каковы некоторые из ваших личных персонаже

In [0]:
dups = []
for i in results:
  for j in i:
    ind = j[0]
    is_dupl_score = np.array(test_table)[ind]
    dups.append(is_dupl_score[3])

In [0]:
s = 0
for i in dups:
  s += i
  
acc = s/len(dups)
acc

BM25

In [0]:
doc_lens = []
for row in test_table['question2']:
  doc_lens.append(len(str(row).split()))

In [0]:
len_mean = np.mean(doc_lens)

In [0]:
corpus = []#массив массивов лемм слов всех текстов
for t in texts:#for text
    ws = []
    if t != "":
      words = t.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      corpus.append(ws)

In [0]:
query_corpus = []#массив массивов лемм слов всех текстов запросов
for q in queries:#for text
    ws = []
    if q != "":
      words = q.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      query_corpus.append(ws)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [0]:
arrstr = []
for text in corpus:  
  s = ' '.join(text)
  arrstr.append(s)

In [0]:
X = vectorizer.fit_transform(arrstr)
matrix = X.toarray()

In [0]:
words = vectorizer.get_feature_names()

In [0]:
tf_matrix = matrix / np.array(doc_lens).reshape((-1, 1))

In [0]:
def tf_matrix_func(tf_matrix, b): 
    k = 2
    pairs =  np.ndenumerate(tf_matrix)
    for i, value in pairs:
        doc_id = i[0]
        l = doc_lens[doc_id]
        tf_matrix[i] = (value * (k + 1.0)) / (value + k * (1.0 - b + b * (l/len_mean)))
    return tf_matrix

In [0]:
from math import log
num_doc_qi = np.count_nonzero(matrix, axis=0) #number of docs with qi

def idf_score(word):
    word_id = words.index(word)
    num = num_doc_qi[word_id]
    score = log((test_N - num + 0.5) / (num + 0.5))
    return score

In [0]:
all_idfs = []
for word in words:
  score = idf_score(word)
  all_idfs.append(score)

In [0]:
b = 0.75

In [0]:
tf_matrix_upd_1 = tf_matrix_func(tf_matrix, b) #матрица со значениями по новой формуле

In [43]:
start2 = time()
for i in range(len(tf_matrix_upd_1)):
  for j in range(len(tf_matrix_upd_1[i])):
    tf_matrix_upd_1[i][j] = tf_matrix_upd_1[i][j] * all_idfs[j] #матрица со значениями bm25 для корпуса
time2 = str(time() - start2)
print('Time of bm25 idx matrix: ', time2)

Time of bm25 idx matrix:  29.11378264427185


In [0]:
def bm25_vect_ver(query):
  
  query_tok = tokenize(query)
  query_new = [' '.join(query_tok)]
  query_vector = np.array(vectorizer.transform(query_new).todense())[0]
  
  scores_bm25 = tf_matrix_upd_1.dot(query_vector)
  
  return scores_bm25

In [0]:
def test(query, b): #функция возвращает 1, если в топ 5 есть хоть однин документ с 1 в is_duplicate, и 0, если такого нет.
  bm25_scores = bm25_vect_ver(query)
  mas = []
  new_list = sorted(enumerate(bm25_scores), key=lambda x:x[1], reverse=True)
  for i in new_list:
    mas.append(i[0])
  top_5 = mas[:5]
  is_dupl_scores = []
  for rank, idx in enumerate(top_5):
    is_dupl_score = np.array(test_table)[idx]
    is_dupl_scores.append(is_dupl_score[3])
  if 1 in is_dupl_scores:
    success = 1
  else:
    success = 0
  return success

In [0]:
def acc_score(b):
  sum_score = 0
  for i in queries[:10]:
    success = test(i, b)
    sum_score += success
  acc = sum_score/len(queries[:10])
  return acc

In [0]:
acc_1 = acc_score(0.75)

In [54]:
acc_1

0.9